In [1]:
pip install groq openai multion boto3 mem0ai

In [2]:
import os
from openai import OpenAI
from mem0 import Memory
from multion.client import MultiOn
import json
import logging
import base64
import boto3
import openai
import agentops
import time
import copy
from groq import Groq

from mem0 import Memory
from botocore.exceptions import ClientError
import os
import requests
import shutil

In [3]:
m = Memory()

In [4]:
groq_client = Groq(
    api_key=os.environ.get("GROQ_API_KEY")
)
client = OpenAI()
client_multion = MultiOn(api_key=os.environ.get("MULTION_API_KEY"), agentops_api_key=os.environ.get("AGENTOPS_API_KEY"))

In [5]:
temperature = 0.0

In [6]:
# user_prompt = """
# Shopping for a dress.
# """

user_prompt = """
Play Lose Yourself by Eminem on Spotify.
"""

In [7]:
system_prompt_llm1_init = """
You are an assistant that will take a web navigation task as an input and output
a detailed chain of thought set of instructions to complete this task. If you
require more information than what is given in the original prompt, your output
should include asking this question as one of the steps. The question should be
formatted using "quotation marks" so that it is easier to follow. After you give a response, you will be given
feedback by another teacher assistant to iteratively improve your response.
Essentially, you are the actor in an actor-critic model. The goal is to
create a set of instructions for another LLM agent to perform using web navigation,
so the response should be framed for an LLM, not a human. For context, the LLM has
access to Google Chrome and maybe other applications on the user's laptop.

Types of questions to ask the user are about:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

This clarification may be needed because the user of this product is not an expert
at prompting or specificity. For example, if the user wants to watch sports clips,
you can ask which sport. Then, if they say soccer, ask if they want recent highlights or
soccer clips from any specific time period. Clarifying user intent is crucial.

RESPONSE SHOULD BE IN LIST FORMAT, WHERE EACH ELEMENT HAS A SHORT RELEVANT TITLE AND CORRESPONDING DESCRIPTION:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: ORIGINAL PROMPT FROM USER ,
<<<TITLE-DESCRIPTION>>>: DESCRIPTION OF STEP ,
<<<ASK-QUESTION>>>: "QUESTION TO ASK TO THE USER OF THIS PRODUCT?" ,
``MORE LINES HERE``
]
"""

system_prompt_llm2_init = """
You are an assistant that will take the proposed chain of thought for completing
a web navigation task and output constructive criticism feedback for the chain
of thought. Your job is to find flaws in the chain of thought logic, misordered
steps, or unnecessary steps. Essentially, you are the critic in an actor-critic model.

Some of the questions that the actor asks the user are crucial, like the following:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

That being said, steps should not be redundant or unnecessary.

INPUT FORMAT:
["{'role': 'user', 'content': 'Initial user prompt here.'}",
 '{\'role\': \'assistant\', \'content\': \'[
  \\n<<<TITLE-OF-STEP>>>: Step description ,
  \\n<<<ASK-QUESTION>>>: "Question to user here (should be as specific as possible)?"
  `MORE-LINES-HERE`'}',']

OUTPUT FORMAT:
Here is some feedback for the chain of thought so far:
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
`MORE-LINES-HERE`
"""

system_prompt_llm1 = """
You are an assistant that will take a web navigation task as an input and output
a detailed chain of thought set of instructions to complete this task.

Complete set of inputs:
1) Initial user prompt
2) Chain of thought produced to complete initial prompt
3) Log messages from an LLM agent trying to complete the web navigation task
based on the given prompt and chain of thought.

Your task is to determine whether the task has been completed or not. If the
task has not been completed, create a chain of thought to complete the task
from the current state. If the task is completed, output: "DONE-WITH-TASK"

If the task is not completed:
If you require more information than what is given in the original prompt, your output
should include asking this question as one of the steps. The question should be
formatted using "quotation marks" so that it is easier to follow. After you give a response, you will be given
feedback by another teacher assistant to iteratively improve your response.
Essentially, you are the actor in an actor-critic model. The goal is to
create a set of instructions for another LLM agent to perform using web navigation,
so the response should be framed for an LLM, not a human. For context, the LLM has
access to Google Chrome and maybe other applications on the user's laptop.

Types of questions to ask the user are about:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

This clarification may be needed because the user of this product is not an expert
at prompting or specificity. For example, if the user wants to watch sports clips,
you can ask which sport. Then, if they say soccer, ask if they want recent highlights or
soccer clips from any specific time period. Clarifying user intent is crucial.

RESPONSE SHOULD BE IN LIST FORMAT, WHERE EACH ELEMENT HAS A SHORT RELEVANT TITLE AND CORRESPONDING DESCRIPTION:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: ORIGINAL PROMPT FROM USER ,
<<<TITLE-DESCRIPTION>>>: DESCRIPTION OF STEP ,
<<<ASK-QUESTION>>>: "QUESTION TO ASK TO THE USER OF THIS PRODUCT?" ,
``MORE LINES HERE``
]
"""

system_prompt_llm2 = """
You are an assistant that will take the proposed chain of thought for completing
a web navigation task and output constructive criticism feedback for the chain
of thought.

Complete set of inputs:
1) Initial user prompt
2) Chain of thought produced to complete initial prompt
3) Log messages from an LLM agent trying to complete the web navigation task
based on the given prompt and chain of thought.
4) A proposed chain of thought to complete the task in the current state.

Your job is to find flaws in the proposed chain of thought logic, misordered
steps, or unnecessary steps. Essentially, you are the critic in an actor-critic model.

Some of the questions that the actor asks the user are crucial, like the following:
1) clarify specificity of the inputted prompt
2) clarify the intent of the inputted prompt
3) credentials to log into websites (usernames and passwords)
(Please assume the user has an account they ask about unless specified otherwise.
Ask them for the credentials, not whether they have the account.)

That being said, steps should not be redundant or unnecessary.

INPUT FORMAT FOR PROPOSED CHAIN OF THOUGHT:
["{'role': 'user', 'content': 'Initial user prompt here.'}",
 '{\'role\': \'assistant\', \'content\': \'[
  \\n<<<TITLE-OF-STEP>>>: Step description ,
  \\n<<<ASK-QUESTION>>>: "Question to user here (should be as specific as possible)?"
  `MORE-LINES-HERE`'}',']

OUTPUT FORMAT:
Here is some feedback for the chain of thought so far:
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
<<<TITLE-OF-FEEDBACK>>>: DESCRIPTION OF FEEDBACK ,
`MORE-LINES-HERE`
"""

bedrock = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)
info = boto3.client(
    service_name="bedrock",
    region_name="us-east-1"
)
llm_3_prompt = """
  You are an agent that is working with a tool called MultiOn to perform google search queries for the user.
  However, MultiOn often makes mistakes and can get sidetracked and navigate to a page that was not what the user intended.
  Additionally, MultiOn can get stuck in a loop where it doesn't know what to do. Given a screenshot of the current page, the
  user prompt which has been processed by two LLMs in a Chain-of-Thought style conversation, and descriptions of the action MultiOn is currently
  taking, you should output either CONTINUE if you think MultiOn is on the right track, STOP_LLM + the reason why MultiOn is not on the right track if you think MultiOn
  is not following the initial user's intent and that MultiOn can be helped with a few clarifications, STOP_HUMAN if MultiOn is unable to use some aspect of the UI such as buttons
  and needs the user's help or if the user's intent is unclear or vague, and END if MultiOn has completed the task that the user asked for.

  Once again, response should be in a specific format:

  1) <<<CONTINUE>>>: <<<N/A>>> (reasoning: multion is on the right track)

  2) <<<STOP_LLM>>>: <<<REASON>>> (reasoning: multion is veering from the user's intentions and moving in the wrong direction)

  3) <<<STOP_HUMAN>>>: <<<REASON>>> (reasoning: multion is stuck or the user's prompt is too vague and more info is needed)

  4) <<<END>>>: <<<N/A>>> (reasoning: the task that the user specified has been completed)
  Generate no other text, and format it exactly in one of the ways shown.
"""
model_id = "anthropic.claude-3-sonnet-20240229-v1:0"

In [8]:
def create_loop(user_prompt, initial_url, system_prompt, max_steps=None):
  step = 0
  messages = []
  create_response = client_multion.sessions.create(
    url=initial_url
  )
  session_id = create_response.session_id
  status = "CONTINUE"
  while status == "CONTINUE":
      step_response = client_multion.sessions.step(
          session_id=session_id,
          cmd=user_prompt,
          include_screenshot=True
      )
      status = step_response.status

      file_name = "test{num}.png"
      time.sleep(3)
      ss = requests.get(step_response.screenshot)
      with open(file_name.format(num=step),'wb') as f:
        f.write(ss.content)
      response = call_claude(messages, step_response.message, file_name.format(num=step))
      step += 1

      if(step >= max_steps):
        break
  return messages, session_id

In [9]:
logger = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)

def run_multi_modal_prompt(bedrock_runtime, model_id, messages, max_tokens):
    """
    Invokes a model with a multimodal prompt.
    Args:
        bedrock_runtime: The Amazon Bedrock boto3 client.
        model_id (str): The model ID to use.
        messages (JSON) : The messages to send to the model.
        max_tokens (int) : The maximum  number of tokens to generate.
    Returns:
        None.
    """
    body = json.dumps(
        {
            "anthropic_version": "bedrock-2023-05-31",
            "system": llm_3_prompt,
            "max_tokens": max_tokens,
             "messages": messages
        }
    )

    response = bedrock_runtime.invoke_model(
        body=body, modelId=model_id)
    response_body = json.loads(response.get('body').read())

    return response_body

def call_claude(old_messages, input_text, input_image):
    try:
        bedrock_runtime = boto3.client(service_name='bedrock-runtime')
        model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'
        max_tokens = 1000

        # Read reference image from file and encode as base64 strings.
        with open(input_image, "rb") as image_file:
            content_image = base64.b64encode(image_file.read()).decode('utf8')

        message = {
            "role": "user",
            "content": [
              {"type": "image", "source": {"type": "base64",
                  "media_type": "image/jpeg", "data": content_image}},
              {"type": "text", "text": input_text}
            ]
        }

        if(old_messages[-1]["role"] == "user"):
          message["content"][1]["text"] += old_messages[-1]["content"]
          old_messages.pop()
        old_messages.append(message)
        response = run_multi_modal_prompt(
            bedrock_runtime, model_id, old_messages, max_tokens)

        old_messages[-1]["content"] = old_messages[-1]["content"][-1]["text"]
        old_messages.append({"role": "assistant", "content" : response["content"][0]["text"]})
        return response["content"][0]["text"]

    except ClientError as err:
        message = err.response["Error"]["Message"]
        logger.error("A client error occurred: %s", message)
        print("A client error occured: " +
              format(message))

In [10]:
# LLM 1 (Initial Proposal)

def generate_COT(messages, first=True, max_iters=3):
  copy_messages = copy.deepcopy(messages)

  completion = groq_client.chat.completions.create(
    messages=[{"role": "system", "content": system_prompt_llm1_init if first else system_prompt_llm1}] + copy_messages,
    model="llama3-8b-8192",
    temperature=temperature,
  )

  response = completion.choices[0].message.content
  copy_messages.append({"role": "assistant", "content": response})
  print("PROPOSED CHAIN OF THOUGHT:")
  print(response)
  print("\n\n")

  for i in range(max_iters):
    # LLM 2
    completion = groq_client.chat.completions.create(
      model="llama3-8b-8192",
      messages=[{"role": "system", "content": system_prompt_llm2_init if first else system_prompt_llm2}, {"role": "user", "content": "\n".join(list(map(str, [copy_messages[0], copy_messages[-1]])))}],
      temperature=temperature,
    )

    response = completion.choices[0].message.content
    copy_messages.append({"role": "user", "content": response})
    # print("FEEDBACK")
    # print(response)
    # print("\n\n")

    # LLM 1
    completion = groq_client.chat.completions.create(
      model="llama3-8b-8192",
      messages=[{"role": "system", "content": system_prompt_llm1_init}] + copy_messages,
      temperature=temperature,
    )

    response = completion.choices[0].message.content
    copy_messages.append({"role": "assistant", "content": response})
    # print("PROPOSED CHAIN OF THOUGHT:")
    # print(response)
    # print("\n\n")

  final_prompt = copy_messages[-1]["content"]
  return final_prompt

In [11]:
def create_full_loop(user_prompt, initial_url="https://google.com/", max_steps=None):
  step = 0
  create_response = client_multion.sessions.create(
    url=initial_url
  )
  messages = [{"role":"user", "content": user_prompt}]
  multion_prompt = generate_COT(messages)
  messages[0]["content"] = multion_prompt
  session_id = create_response.session_id
  status = "CONTINUE"
  while status != "DONE":
      step_response = client_multion.sessions.step(
          session_id=session_id,
          cmd=multion_prompt,
          include_screenshot=True
      )

      m.add(step_response.message, user_id="multion", metadata={"category":"MultiOn-Message"})
      m.add(step_response.screenshot, user_id="multion", metadata={"category":"MultiOn-Screenshot"})
      print("\n\n MultiOn Message: " + step_response.message)
      print("MultiOn Screenshot: " + step_response.screenshot)
      print("MultiOn Status: " + step_response.status)
      status = step_response.status

      file_name = "test{num}.png"
      time.sleep(3)
      ss = requests.get(step_response.screenshot)
      with open(file_name.format(num=step),'wb') as f:
        f.write(ss.content)


      if(status == "NOT_SURE"):
        temp_input = input()
        if(messages[-1]["role"] == "user"):
          messages[-1]["content"] += temp_input
        else:
          messages.append({"role":"user", "content": temp_input})

        if(len(temp_input) > 0):
          multion_prompt = temp_input

      else:
        response = call_claude(messages, step_response.message, file_name.format(num=step))
        m.add(response, user_id = "claude", metadata={"category":"Claude-Response"})
        print("\n\n Claude Response: " + response)
        if(status == "DONE" and not ("STOP_LLM" in response or "STOP_HUMAN" in response)):
          status = "CONTINUE"
        if("CONTINUE" in response):
          pass
        elif("STOP_LLM" in response):
          multion_prompt = generate_COT(messages, first=False)
          multion_prompt = multion_prompt + response

          if(messages[-1]["role"] == "user"):
            messages[-1]["content"] += multion_prompt
          else:
            messages.append({"role":"user", "content": multion_prompt})
        elif("STOP_HUMAN" in response):
          temp_input = input()
          if(messages[-1]["role"] == "user"):
            messages[-1]["content"] += temp_input
          else:
            messages.append({"role":"user", "content": temp_input})
          if(len(temp_input) > 0):
            multion_prompt = temp_input

          multion_prompt = multion_prompt + response
        else:
          print("3rd LLM says task completed!")
          break
      step += 1
      if(max_steps and step >= max_steps):
        break
  return messages, session_id

In [ ]:
user_prompt = "Tell me the weight of the sony xm4 and find other headphones with a similar weight"
messages, session_id = create_full_loop(user_prompt)

🖇 AgentOps: Session Replay: https://app.agentops.ai/drilldown?session_id=b21d8638-f953-43c2-9f8e-98507dbb93dd
INFO:agentops:Session Replay: https://app.agentops.ai/drilldown?session_id=b21d8638-f953-43c2-9f8e-98507dbb93dd


PROPOSED CHAIN OF THOUGHT:
Here is the chain of thought to complete the task:

[
"Original-Prompt-From-User": Find the weight of the Sony XM4 and similar headphones ,
"Step 1: Check Official Website": Check the official Sony website for the weight of the Sony XM4 headphones ,
"Ask-Question": "Can you please confirm if you want to check the weight of the Sony XM4 headphones from the official Sony website or a third-party source?" ,
"Step 2: Check Official Website": If the user confirms, check the official Sony website for the weight of the Sony XM4 headphones. If not, proceed to Step 3 ,
"Step 3: Check Third-Party Source": Check a reputable third-party source such as CNET, Rtings, or Wirecutter for the weight of the Sony XM4 headphones ,
"Step 4: Find Similar Headphones": Search for other headphones with a similar weight to the Sony XM4. You can use search engines like Google or online marketplaces like Amazon ,
"Ask-Question": "What is your preferred price range for the similar headpho

In [ ]:
all_memories = m.get_all()
print(all_memories)

In [ ]:
while messages[-1]["role"] != "assistant" or "DONE-WITH-TASK" not in messages[-1]["content"]:
    # LLM 1
    completion = client.chat.completions.create(
      model="gpt-4o-mini",
      messages=[{"role": "system", "content": system_prompt_llm1}] + messages,
      temperature=temperature,
    )

    response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": response})
    print("PROPOSED CHAIN OF THOUGHT:")
    print(response)
    print("\n\n")

    for i in range(3):
      # LLM 2
      completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt_llm2}, {"role": "user", "content": "\n".join(list(map(str, [messages[0], "Proposed chain of thought to solve task from current state:", messages[-1]])))}],
        temperature=temperature,
      )

      response = completion.choices[0].message.content
      messages.append({"role": "user", "content": response})
      print("FEEDBACK")
      print(response)
      print("\n\n")

      # LLM 1
      completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "system", "content": system_prompt_llm1}] + messages,
        temperature=temperature,
      )

      response = completion.choices[0].message.content
      messages.append({"role": "assistant", "content": response})
      print("PROPOSED CHAIN OF THOUGHT:")
      print(response)
      print("\n\n")

    final_prompt = messages[-1]["content"]

    current_prompt = final_prompt
    step_response = client_multion.sessions.step(
        session_id=create_response.session_id,
        cmd=current_prompt,
        include_screenshot=True
    )
    step_responses.append(step_response)
    status = step_response.status
    print(step_response)

    for i in range(max_iters-1):
        if status not in ["CONTINUE", "DONE"]:
            temp_input = input()
            current_prompt = current_prompt if len(temp_input) == 0 else temp_input
            step_response = client_multion.sessions.step(
                session_id=create_response.session_id,
                cmd=current_prompt,
                include_screenshot=True
            )
        elif status == "DONE":
            break
        elif status == "CONTINUE":
            step_response = client_multion.sessions.step(
                session_id=create_response.session_id,
                cmd=current_prompt,
                include_screenshot=True
            )
        step_responses.append(step_response)
        status = step_response.status
        print(step_response)

PROPOSED CHAIN OF THOUGHT:
[
<<<ORIGINAL-PROMPT-FROM-USER>>>: Shopping for a dress ,
<<<CONFIRM-VERIFICATION>>>: The LLM has reached a point where eBay is asking for verification to proceed with the purchase. The user needs to complete this verification step on their end. ,
<<<ASK-QUESTION>>>: "Please complete the verification process on eBay and let me know once it's done so I can continue assisting you with the purchase." ,
<<<WAIT-FOR-USER-RESPONSE>>>: Wait for the user to confirm that they have completed the verification process before proceeding with the next steps. ,
<<<PROCEED-TO-PURCHASE>>>: Once the user confirms the verification is complete, proceed to add the selected dress to the cart and guide the user through the checkout process. ,
<<<REQUEST-CREDENTIALS-IF-NEEDED>>>: If the user has not provided their eBay credentials yet, ask for their username and password to complete the purchase. ,
<<<ASK-QUESTION>>>: "Please provide your username and password for eBay if you intend

KeyboardInterrupt: 